In [1]:
import wandb

    
api = wandb.Api()
project = "feas-ft"
workspace = "username"

# Get our two main experiments so far
experiment_tags = ["alpaca"]

# get all runs that both: 1.  match any experiment tag and 2. are finished
runs = api.runs(f"{workspace}/{project}",
                {"$and": [
                    {"tags": {"$in": experiment_tags}},
                ]})


In [2]:
#for run in runs:
    #print(run.name) 

In [3]:
import os
import lightning as L
from lit_gpt.args import IOArgs
from lit_gpt.lora import GPT,  Config
from lit_gpt.utils import (
    check_valid_checkpoint_dir,
    lazy_load,
)
from finetune.lora import get_longest_seq_length
from eval.lm_eval_harness import EvalHarnessBase
from pathlib import Path

def setup(run, checkpoints ="checkpoints/meta-llama/Llama-2-7b-hf",
           adapter_path = "downloads/", eval_tasks = ["arc_challenge", "piqa", "hellaswag"], num_fewshot=0):
    config = run.config
    io = IOArgs(
        checkpoint_dir=Path(checkpoints),
        out_dir=Path("out/lora/lima/dloads"),
    )
    conf = Config.from_name(
            name=io.checkpoint_dir.name,
            r=config["r"],
            alpha=config["alpha"],
            dropout=config["dropout"],
            to_query=config["to_query"],
            to_key=config["to_key"],
            to_value=config["to_value"],
            to_projection=config["to_projection"],
            to_mlp=config["to_mlp"],
            to_head=config["to_head"],
        )
    fabric = L.Fabric(devices=1, strategy="auto", precision="bf16-true", plugins=None)
    fabric.seed_everything(0)  # same seed for every process to init model (FSDP)
    
    with fabric.init_module(empty_init=(False)):
        model = GPT(conf)
    #tokenizer = Tokenizer(io.checkpoint_dir)
    check_valid_checkpoint_dir(io.checkpoint_dir)

    if fabric.global_rank == 0:
        os.makedirs(io.out_dir, exist_ok=True)

    checkpoint_path = io.checkpoint_dir / "lit_model.pth"
    #print(adapter_path)
    params = {**lazy_load(Path(adapter_path))["model"], **lazy_load(Path(checkpoint_path))}
    #print(params.keys())
    model.load_state_dict(params, strict=True)
    
    if len(eval_tasks)> 0:
        eval_harness = EvalHarnessBase(
            checkpoint_dir=str(io.checkpoint_dir),
            model=model,
        )

    results = eval_harness.run_eval(
        eval_tasks=eval_tasks, num_fewshot=num_fewshot,  no_cache=True, bootstrap_iters=10000
    )
    wandb.init(id=run.id, project=run.project, resume="allow")
    wandb.log(results['results'])
    wandb.finish()




/home/chiche/miniconda3/envs/bab/lib/python3.10/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
/home/chiche/miniconda3/envs/bab/lib/python3.10/site-packages/torch/cuda/__init__.py:628: UserWarning: Can't initialize NVML
  warnings.warn("Can't initialize NVML")


In [4]:
for run in runs:
    print(run)
    weights = run.file("out/lora/alpaca/lit_model_lora_finetuned.pth")
    # create the directory if it doesn't exist
    os.makedirs(f"download/{run.id}", exist_ok=True)
    out = weights.download(f"download/{run.id}", replace=True)
    setup(run, adapter_path=out.name)
    

<Run username/feas-ft/rbe30j1e (finished)>


Seed set to 0
You are using a CUDA device ('NVIDIA RTX A6000') that has Tensor Cores. To properly utilize them, you should set `torch.set_float32_matmul_precision('medium' | 'high')` which will trade-off precision for performance. For more details, read https://pytorch.org/docs/stable/generated/torch.set_float32_matmul_precision.html#torch.set_float32_matmul_precision
Loading model 'checkpoints/meta-llama/Llama-2-7b-hf/lit_model.pth' with {'name': 'Llama-2-7b-hf', 'hf_config': {'org': 'meta-llama', 'name': 'Llama-2-7b-hf'}, 'scale_embeddings': False, 'block_size': 4096, 'vocab_size': 32000, 'padding_multiple': 64, 'padded_vocab_size': 32000, 'n_layer': 32, 'n_head': 32, 'head_size': 128, 'n_embd': 4096, 'rotary_percentage': 1.0, 'parallel_residual': False, 'bias': False, 'lm_head_bias': False, 'n_query_groups': 32, 'shared_attention_norm': False, '_norm_class': 'RMSNorm', 'norm_eps': 1e-05, '_mlp_class': 'LLaMAMLP', 'gelu_approximate': 'none', 'intermediate_size': 11008, 'rope_condense

Found tasks: ['hellaswag', 'arc_challenge', 'piqa']


/home/chiche/miniconda3/envs/bab/lib/python3.10/site-packages/datasets/load.py:1461: FutureWarning: The repository for hellaswag contains custom code which must be executed to correctly load the dataset. You can inspect the repository content at https://hf.co/datasets/hellaswag
You can avoid this message in future by passing the argument `trust_remote_code=True`.
Passing `trust_remote_code=True` will be mandatory to load this dataset from the next major release of `datasets`.
  warnings.warn(


Task: hellaswag; number of docs: 10042
Task: hellaswag; document 0; context prompt (starting on next line):
Personal Care and Style: How to increase breast size with a bra. Check your bra size. Wearing a bra that is too big will not make your breasts look larger. That is why it is important to wear the right size bra for you.
(end of prompt on previous line)
Requests: [Req_loglikelihood('Personal Care and Style: How to increase breast size with a bra. Check your bra size. Wearing a bra that is too big will not make your breasts look larger. That is why it is important to wear the right size bra for you.', ' You can visit a lingerie shop and have them measure you to help you fit a bra to your size, or measure yourself before you shop for a new bra to ensure that you get a good fit. Use a flexible tape measure, like one found in a sewing kit.')[0]
, Req_loglikelihood('Personal Care and Style: How to increase breast size with a bra. Check your bra size. Wearing a bra that is too big will 

...

Found tasks: ['hellaswag', 'arc_challenge', 'piqa']


/home/chiche/miniconda3/envs/bab/lib/python3.10/site-packages/datasets/load.py:1461: FutureWarning: The repository for hellaswag contains custom code which must be executed to correctly load the dataset. You can inspect the repository content at https://hf.co/datasets/hellaswag
You can avoid this message in future by passing the argument `trust_remote_code=True`.
Passing `trust_remote_code=True` will be mandatory to load this dataset from the next major release of `datasets`.
  warnings.warn(


Task: hellaswag; number of docs: 10042
Task: hellaswag; document 0; context prompt (starting on next line):
Personal Care and Style: How to increase breast size with a bra. Check your bra size. Wearing a bra that is too big will not make your breasts look larger. That is why it is important to wear the right size bra for you.
(end of prompt on previous line)
Requests: [Req_loglikelihood('Personal Care and Style: How to increase breast size with a bra. Check your bra size. Wearing a bra that is too big will not make your breasts look larger. That is why it is important to wear the right size bra for you.', ' You can visit a lingerie shop and have them measure you to help you fit a bra to your size, or measure yourself before you shop for a new bra to ensure that you get a good fit. Use a flexible tape measure, like one found in a sewing kit.')[0]
, Req_loglikelihood('Personal Care and Style: How to increase breast size with a bra. Check your bra size. Wearing a bra that is too big will 

100%|██████████| 48508/48508 [48:21<00:00, 16.72it/s]


multiplier/frac_zeros,0.53468
multiplier/max,17.41353
multiplier/mean,0.55879
multiplier/mean_non_zeros,1.20087
multiplier/min,0
multiplier/std,0.9324
slack/frac_zeros,0.60546
slack/max,1.02763
slack/mean,0.02859
slack/mean_non_zeros,0.07245
slack/min,0
